In [1]:
import pandas as pd
import os
import re

In [2]:
detector_dicts = []
detector_descriptor_dicts = []

In [3]:
def process_detector(file_name):
    
    keypoints_performance_dicts = []
    
    with open(file_name, "r") as fin:
        lines = fin.readlines()
    
    base = file_name.split("/")[-1].split(".")[0].lower()
    
    found = False
    
    for line in lines:
        if line.lower().replace("-","").find(base) != -1:
            found = True
            
    if not found:
        print(f"Error {file_name}")
        return
    
    for line in lines:
        result = re.search("\sn=\d+\s",line)
        if result is not None:
            keypoints = int(re.search("\d+", result[0])[0])
        else:
            continue

        result = re.search("\d+.\d+ ms",line)
        if result is not None:
            duration = float(re.search("\d+.\d+", result[0])[0])
            
        keypoints_performance_dicts.append({
            'detector': base,
            'keypoints': keypoints,
            'duration': duration
        })
        
    return keypoints_performance_dicts
    

In [4]:
def process_detector_descriptor(file_name):
    
    rtn_dicts = []
    
    with open(file_name, "r") as fin:
        lines = fin.readlines()
    
    base = file_name.split("/")[-1].split(".")[0]
    detector = base.split("-")[0].lower()
    descriptor = base.split("-")[1].lower()
    
    found_detector = False
    found_descriptor = False
    
    
    for line in lines:
        if line.lower().replace("-","").find(detector) != -1:
            found_detector = True
        if line.lower().replace("-","").find(descriptor) != -1:
            found_descriptor = True
            
    if not found_detector:
        print(f"Did not Find Detector: {file_name}")
    
    if not found_descriptor:
        print(f"Did not Find Descriptor: {file_name}")
        
    if not (found_descriptor and found_detector):
        return
    
    keypoints = -1
    detection_duration = -1
    description_duration = -1
    matches = -1
    
    for line in lines:
        if line.find("LOAD IMAGE INTO BUFFER done") != -1:
            keypoints = -1
            detection_duration = -1
            description_duration = -1
            matches = -1
            continue
        
        result = re.search("\sn=\d+\s",line)
        if result is not None:
            keypoints = int(re.search("\d+", result[0])[0])

        result = re.search("keypoints in \d+.\d+ ms",line)
        if result is not None:
            detection_duration = float(re.search("\d+.\d+", result[0])[0])
            
        result = re.search("extraction in \d+.\d+ ms",line)
        if result is not None:
            description_duration = float(re.search("\d+.\d+", result[0])[0])
            
        result = re.search("Matched \d+",line)
        if result is not None:
            matches = int(re.search("\d+", result[0])[0])
        
        if (keypoints != -1) and (detection_duration != -1) and (description_duration != -1) and (matches != -1):
            rtn_dicts.append({
                'detector': detector,
                'descriptor': descriptor,
                'keypoints': keypoints,
                'detector-duration': detection_duration,
                'descriptor-duration': description_duration,
                'matches': matches,

            })
            keypoints = -1
            detection_duration = -1
            description_duration = -1
            matches = -1
            
    return rtn_dicts
            
        

In [5]:
for root, dirs, files in os.walk("./experiments/"):
    for file in files:
        file_name = os.path.join(root, file)
        
        if file_name.find("-") == -1:
            results = process_detector(file_name)
            for r in results:
                detector_dicts.append(r)
        else:
            results = process_detector_descriptor(file_name)
            for r in results:
                detector_descriptor_dicts.append(r)

In [6]:
pd.DataFrame.from_dict(detector_dicts).to_csv("detector_tests.csv", index = False)

In [7]:
pd.DataFrame.from_dict(detector_descriptor_dicts).to_csv("detector_descriptor_experiments.csv", index = False)